<a href="https://colab.research.google.com/github/saadkhi/Side/blob/main/Script1_for_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas

In [ ]:
# import json
# import pandas as pd
# from collections import Counter

# # Step 1: Read JSONL file
# def read_jsonl(file_path):
#     data = []
#     with open(file_path, 'r', encoding='utf-8') as f:
#         for line in f:
#             try:
#                 data.append(json.loads(line.strip()))
#             except json.JSONDecodeError as e:
#                 print(f"Error parsing line: {line.strip()} - {e}")
#     return data

# # Step 2: Verify dataset
# def check_dataset(data):
#     print(f"Total rows: {len(data)}")

#     # Check for missing keys
#     for i, entry in enumerate(data, 1):
#         if "prompt" not in entry or "completion" not in entry:
#             print(f"Missing keys in row {i}: {entry}")

#     # Check balance of positive vs. negative examples
#     completions = [entry["completion"] for entry in data]
#     rejection_count = sum(1 for c in completions if "Sorry, I can only answer" in c)
#     print(f"Positive (SQL/NoSQL) examples: {len(data) - rejection_count}")
#     print(f"Negative (unrelated) examples: {rejection_count}")

#     # Check for duplicates
#     prompts = [entry["prompt"] for entry in data]
#     duplicates = [prompt for prompt, count in Counter(prompts).items() if count > 1]
#     if duplicates:
#         print(f"Duplicate prompts found: {duplicates}")
#     else:
#         print("No duplicate prompts found.")

# # Step 3: Create DataFrame
# def create_dataframe(data):
#     # Convert to DataFrame
#     df = pd.DataFrame(data)

#     # Add is_related column: True if completion doesn't contain rejection message
#     df['is_related'] = ~df['completion'].str.contains("Sorry, I can only answer")

#     return df

# # Step 4: Analyze DataFrame
# def analyze_dataframe(df):
#     print("\nDataFrame Info:")
#     df.info()

#     print("\nFirst 5 rows:")
#     df.head()

#     print("\nDistribution of is_related:")
#     df['is_related'].value_counts()

#     # Check for missing values
#     print("\nMissing values:")
#     df.isnull().sum()

# # Main execution
# file_path = "dataset.jsonl"  # Update with your file path
# data = read_jsonl(file_path)
# check_dataset(data)
# df = create_dataframe(data)
# analyze_dataframe(df)

# # Optional: Save DataFrame to CSV for further use
# df.to_csv("database_dataset.csv", index=False)
# print("\nDataFrame saved as 'database_dataset.csv'")

In [ ]:
import json

# Read file line by line to find invalid JSON
with open('dataset.jsonl', 'r') as f:
    for i, line in enumerate(f, 1):
        try:
            json.loads(line.strip())  # Try parsing each line
        except json.JSONDecodeError as e:
            print(f"Error in line {i}: {line.strip()} - {e}")

In [ ]:
import pandas as pd

# Load the corrected JSONL file
try:
    df = pd.read_json('dataset.jsonl', lines=True)
    print("DataFrame loaded successfully!")
except ValueError as e:
    print(f"Still an error: {e}. Please check the JSONL file for formatting issues.")

DataFrame loaded successfully!


In [ ]:
df.head()

,prompt,completion
0,Write a SQL query to find all employees with s...,SELECT * FROM employees WHERE salary > 50000;
1,How do I join two tables in SQL to get custome...,"SELECT c.customer_id, c.name, o.order_id FROM ..."
2,Explain the difference between INNER JOIN and ...,INNER JOIN returns only matching rows from bot...
3,Write a SQL query to get the top 5 products by...,"SELECT product_name, SUM(sales) as total_sales..."
4,How to create a table in MySQL for storing use...,CREATE TABLE users (id INT AUTO_INCREMENT PRIM...


In [ ]:
df.shape

(6030, 2)

In [ ]:
# Check for missing or empty values
missing_prompts = df[df['prompt'].isnull() | (df['prompt'] == '')]
missing_completions = df[df['completion'].isnull() | (df['completion'] == '')]
print("Missing/empty prompts:", len(missing_prompts))
print("Missing/empty completions:", len(missing_completions))
print("Missing prompts sample:\n", missing_prompts[['prompt', 'completion']].head())
print("Missing completions sample:\n", missing_completions[['prompt', 'completion']].head())

Missing/empty prompts: 0
Missing/empty completions: 0
Missing prompts sample:
 Empty DataFrame
Columns: [prompt, completion]
Index: []
Missing completions sample:
 Empty DataFrame
Columns: [prompt, completion]
Index: []


In [ ]:
# Check for duplicate prompts
duplicates = df[df.duplicated(subset=['prompt'], keep=False)]
print("Duplicate prompts count:", len(duplicates))
print("Duplicate prompts sample:\n", duplicates[['prompt', 'completion']].head())

Duplicate prompts count: 3475
Duplicate prompts sample:
                                                prompt  \
0   Write a SQL query to find all employees with s...   
2   Explain the difference between INNER JOIN and ...   
12    Write a SQL query to find second-highest salary   
13                      How to use a subquery in SQL?   
20                    Write a SQL query to pivot data   

                                           completion  
0       SELECT * FROM employees WHERE salary > 50000;  
2   INNER JOIN returns only matching rows from bot...  
12  SELECT MAX(salary) FROM employees WHERE salary...  
13  A subquery is a query nested inside another, e...  
20  In PostgreSQL, use CROSSTAB or CASE: SELECT de...  


In [ ]:
# Check prompt and completion lengths for outliers
df['prompt_length'] = df['prompt'].str.len()
df['completion_length'] = df['completion'].str.len()
short_prompts = df[df['prompt_length'] < 10]
long_prompts = df[df['prompt_length'] > 2000]
short_completions = df[df['completion_length'] < 10]
long_completions = df[df['completion_length'] > 2000]
print("Short prompts (<10 chars):", len(short_prompts))
print("Long prompts (>2000 chars):", len(long_prompts))
print("Short completions (<10 chars):", len(short_completions))
print("Long completions (>2000 chars):", len(long_completions))
print("Short prompts sample:\n", short_prompts[['prompt', 'completion']].head())

Short prompts (<10 chars): 0
Long prompts (>2000 chars): 0
Short completions (<10 chars): 31
Long completions (>2000 chars): 0
Short prompts sample:
 Empty DataFrame
Columns: [prompt, completion]
Index: []


In [ ]:
# Check for inconsistent refusal messages
refusal_pattern = "Sorry, I can only answer questions related to SQL, NoSQL, and the database domain."
inconsistent_refusals = df[(df['completion'].str.startswith("Sorry", na=False) & (df['completion'] != refusal_pattern))]
print("Inconsistent refusals count:", len(inconsistent_refusals))
print("Inconsistent refusals sample:\n", inconsistent_refusals[['prompt', 'completion']].head())

Inconsistent refusals count: 1072
Inconsistent refusals sample:
                               prompt  \
68         What’s the weather today?   
69  How to code a website in Python?   
70     What’s the capital of France?   
71   Explain machine learning basics   
72         Query the meaning of life   

                                           completion  
68  Sorry, I can only answer questions related to ...  
69  Sorry, I can only answer questions related to ...  
70  Sorry, I can only answer questions related to ...  
71  Sorry, I can only answer questions related to ...  
72  Sorry, I can only answer questions related to ...  


In [ ]:
# Basic check for potentially invalid SQL/NoSQL completions (approximate)
positive_examples = df[~df['completion'].str.startswith("Sorry", na=False)]
missing_sql_keywords = positive_examples[
    positive_examples['completion'].str.contains('SELECT|INSERT|UPDATE|DELETE|CREATE|ALTER|DROP', case=False, na=False) == False
]
missing_nosql_keywords = positive_examples[
    positive_examples['completion'].str.contains('db.collection|find|aggregate|insert|update|delete|KEYS|HSET|SMEMBERS|UPDATE keyspace', case=False, na=False) == False
]
print("Potential invalid SQL completions (no common keywords):", len(missing_sql_keywords))
print("Potential invalid NoSQL completions (no common keywords):", len(missing_nosql_keywords))
print("Potential invalid SQL sample:\n", missing_sql_keywords[['prompt', 'completion']].head())

Potential invalid SQL completions (no common keywords): 2836
Potential invalid NoSQL completions (no common keywords): 3590
Potential invalid SQL sample:
                                                prompt  \
2   Explain the difference between INNER JOIN and ...   
6                        What’s a primary key in SQL?   
8                   How to handle NULL values in SQL?   
10  What’s the difference between GROUP BY and ORD...   
19  What’s the difference between SQL and NoSQL da...   

                                           completion  
2   INNER JOIN returns only matching rows from bot...  
6   A primary key is a unique, non-null column (or...  
8   Use IS NULL or IS NOT NULL to check for NULLs,...  
10  GROUP BY aggregates rows into groups based on ...  
19  SQL databases are relational, use structured s...  


# **MODEL FINETUNING**

In [ ]:
!pip install transformers peft datasets torch

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load dataset
df = pd.read_json('dataset.jsonl', lines=True)

# Split into train (80%), validation (10%), test (10%)
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)
print("Train:", len(train_df), "Val:", len(val_df), "Test:", len(test_df))

# Save splits
train_df.to_json('train.jsonl', orient='records', lines=True)
val_df.to_json('val.jsonl', orient='records', lines=True)
test_df.to_json('test.jsonl', orient='records', lines=True)

Train: 4824 Val: 603 Test: 603


In [ ]:
from transformers import AutoTokenizer
from datasets import Dataset

# Load tokenizer (replace with your model, e.g., "distilgpt2" or "meta-llama/Llama-2-7b-hf")
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token  # Add padding token if needed

# Load datasets
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

# Tokenize function
def tokenize_function(examples):
    inputs = tokenizer(examples['prompt'], padding="max_length", truncation=True, max_length=512)
    outputs = tokenizer(examples['completion'], padding="max_length", truncation=True, max_length=512)
    inputs['labels'] = outputs['input_ids']
    return inputs

# Apply tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
val_dataset = val_dataset.map(tokenize_function, batched=True)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/4824 [00:00<?, ? examples/s]

Map:   0%|          | 0/603 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForCausalLM
from peft import LoraConfig, get_peft_model

# Load model (already done, but included for clarity)
model_name = "distilgpt2"
model = AutoModelForCausalLM.from_pretrained(model_name)

# Configure LoRA with correct target modules for DistilGPT-2
lora_config = LoraConfig(
    r=8,  # Rank of adaptation
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["c_fc", "c_proj"],  # Correct modules for DistilGPT-2's feed-forward layers
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)
print("LoRA model configured successfully!")

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

LoRA model configured successfully!


/usr/local/lib/python3.12/dist-packages/peft/tuners/lora/layer.py:2174: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


In [ ]:
from transformers import TrainingArguments, Trainer

# Define training arguments
training_args = TrainingArguments(
    output_dir="./fine_tuned_model",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=2e-5,
    eval_strategy="steps",
    eval_steps=500,
    save_steps=500,
    logging_steps=100,
    save_total_limit=2,
    fp16=True,  # Enable if GPU supports it
)

# Initialize trainer (use 'model' from Step 4, not 'model_name')
trainer = Trainer(
    model=model,  # Corrected: use the model object, not model_name
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset
)

# Train
trainer.train()

# Save model
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")
print("Training completed and model saved!")

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: saadalioffic (saadalioffic-usman-institute-of-technology-uit) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss
500,0.376300,0.369623
1000,0.336100,0.301403
1500,0.294000,0.277252
2000,0.270200,0.266858
2500,0.284500,0.261666
3000,0.254000,0.261158
3500,0.252700,0.258690


Training completed and model saved!


In [ ]:
from datasets import Dataset
import torch

# Load test dataset
test_dataset = Dataset.from_pandas(pd.read_json('test.jsonl', lines=True))
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Evaluate on test set
results = trainer.evaluate(test_dataset)
print("Test set evaluation:", results)

# Manual inspection (check 5 examples)
for i in range(5):
    input_text = test_df.iloc[i]['prompt']
    expected = test_df.iloc[i]['completion']
    inputs = tokenizer(input_text, return_tensors="pt", padding=True, truncation=True)
    # Move inputs to CUDA (same device as model)
    inputs = {k: v.to('cuda') for k, v in inputs.items()}
    # Ensure model is on CUDA (should already be, but confirm)
    model.to('cuda')
    outputs = model.generate(**inputs, max_length=512)
    predicted = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(f"Prompt: {input_text}\nExpected: {expected}\nPredicted: {predicted}\n")

Map:   0%|          | 0/603 [00:00<?, ? examples/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Test set evaluation: {'eval_loss': 0.25815361738204956, 'eval_runtime': 10.0459, 'eval_samples_per_second': 60.024, 'eval_steps_per_second': 15.031, 'epoch': 3.0}


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: What's the history of Rome?
Expected: Sorry, I can only answer questions related to SQL and NoSQL databases.
Predicted: What's the history of Rome?

Prompt: Explain database denormalization.
Expected: Denormalization intentionally duplicates data to improve read performance, common in analytics or NoSQL.
Predicted: Explain database denormalization.

Prompt: Write a Neo4j query to update a node.
Expected: MATCH (n:Person {name: 'Alice'}) SET n.age = 31;
Predicted: Write a Neo4j query to update a node.

Prompt: Write a SQL query to join three tables
Expected: SELECT * FROM table1 JOIN table2 ON table1.id = table2.id JOIN table3 ON table2.id = table3.id;
Predicted: Write a SQL query to join three tables

Prompt: Explain database transactions.
Expected: Transactions ensure a series of operations complete atomically or roll back on failure.
Predicted: Explain database transactions.



In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load fine-tuned model and tokenizer
model_path = "./fine_tuned_model"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Ensure model is on CUDA (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Get custom prompt from user
custom_prompt = input("Enter your prompt: ")

# Tokenize prompt
inputs = tokenizer(custom_prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Move inputs to the same device as model
inputs = {k: v.to(device) for k, v in inputs.items()}

# Generate response
outputs = model.generate(**inputs, max_length=512, num_return_sequences=1)

# Decode and print response
predicted = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(f"Prompt: {custom_prompt}\nPredicted: {predicted}")

Enter your prompt: Write a SQL query to find employees by salary range.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Prompt: Write a SQL query to find employees by salary range.
Predicted: Write a SQL query to find employees by salary range.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import pandas as pd

# Load fine-tuned model and tokenizer
model_path = "./fine_tuned_model"
model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Ensure model is on CUDA (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Get custom prompt from user
custom_prompt = input("Enter your prompt: ")

# Format prompt for inference (mimic training format)
formatted_prompt = f"Prompt: {custom_prompt} Response:"

# Tokenize prompt
inputs = tokenizer(formatted_prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Move inputs to the same device as model
inputs = {k: v.to(device) for k, v in inputs.items()}

# Generate response with adjusted parameters
outputs = model.generate(
    **inputs,
    max_length=512,
    num_return_sequences=1,
    temperature=0.7,  # Encourage diverse outputs
    top_p=0.9,        # Use nucleus sampling
    do_sample=True,   # Enable sampling to avoid repetition
    pad_token_id=tokenizer.eos_token_id
)

# Decode and print response
predicted = tokenizer.decode(outputs[0], skip_special_tokens=True)
# Extract response after "Response:" if present
predicted = predicted.replace(formatted_prompt, "").strip()
print(f"Prompt: {custom_prompt}\nPredicted: {predicted}")

# Quick dataset check: Look for similar prompts
df = pd.read_json('dataset.jsonl', lines=True)
similar_prompts = df[df['prompt'].str.contains("merge.*columns", case=False, na=False)]
print(f"Found {len(similar_prompts)} similar prompts in dataset:")
print(similar_prompts[['prompt', 'completion']].head())

Enter your prompt: Write a SQL query to find employees by salary range.
Prompt: Write a SQL query to find employees by salary range.
Predicted: 
Found 0 similar prompts in dataset:
Empty DataFrame
Columns: [prompt, completion]
Index: []


In [ ]:
import pandas as pd
df = pd.read_json('dataset.jsonl', lines=True)
print(df.columns)  # Should show ['prompt', 'completion']

Index(['prompt', 'completion'], dtype='object')


In [ ]:
non_strings = df[df['completion'].apply(lambda x: not isinstance(x, str))]
print("Non-string completions:", len(non_strings))
print(non_strings.head())

Non-string completions: 0
Empty DataFrame
Columns: [prompt, completion]
Index: []


In [ ]:
df['completion'] = df['completion'].apply(lambda x: str(x) if pd.notnull(x) else "Sorry, I can only answer questions related to SQL and NoSQL databases.")
df = df.dropna(subset=['prompt', 'completion'])  # Drop any remaining nulls

In [ ]:
positives = df[df['completion'].str.startswith("Sorry") == False]
negatives = df[df['completion'].str.startswith("Sorry")]
print("Positives:", len(positives), "Negatives:", len(negatives))

Positives: 8193 Negatives: 1815


In [ ]:
from sklearn.model_selection import train_test_split
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)
train_df.to_json('train.jsonl', orient='records', lines=True)
val_df.to_json('val.jsonl', orient='records', lines=True)
test_df.to_json('test.jsonl', orient='records', lines=True)
print("Train:", len(train_df), "Val:", len(val_df), "Test:", len(test_df))

Train: 8006 Val: 1001 Test: 1001


In [ ]:
!pip install huggingface_hub
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `Querybot` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-cred

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import os

# Load dataset
dataset = load_dataset("json", data_files={"train": "train.jsonl", "validation": "val.jsonl"})

# Tokenize for chat format
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mixtral-7B-v0.1")
tokenizer.pad_token = tokenizer.eos_token
def tokenize(examples):
    messages = [{"role": "user", "content": p} for p in examples['prompt']]
    formatted = [tokenizer.apply_chat_template([msg, {"role": "assistant", "content": c}], tokenize=False)
                for msg, c in zip(messages, examples['completion'])]
    return tokenizer(formatted, padding="max_length", truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize, batched=True)

# Load model with LoRA
model = AutoModelForCausalLM.from_pretrained("mistralai/Mixtral-7B-v0.1", device_map="auto")
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj", "k_proj", "o_proj"],
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

# Train
args = TrainingArguments(
    output_dir="./fine_tuned_model",
    num_train_epochs=4,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=5e-6,
    eval_strategy="steps",
    eval_steps=500,
    save_steps=500,
    fp16=True,
    report_to="none"  # Or "wandb" for logging
)
trainer = Trainer(model=model, args=args, train_dataset=tokenized_datasets["train"], eval_dataset=tokenized_datasets["validation"])
trainer.train()

# Save
model.save_pretrained("./fine_tuned_model")
tokenizer.save_pretrained("./fine_tuned_model")

OSError: mistralai/Mixtral-7B-v0.1 is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `hf auth login` or by passing `token=<your_token>`